In [600]:
import random
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from string import punctuation
import json, os
from collections import Counter
import numpy as np
import warnings
import nltk
from nltk.stem import WordNetLemmatizer

In [601]:
warnings.filterwarnings('ignore')

punct = punctuation+"«»—…“”*№–'"
stops = set(stopwords.words('english'))

def digits(string): 
    return any(char.isdigit() for char in string)

wordnet_lemmatizer = WordNetLemmatizer()

breaks = ['breaks', 'broken', 'breaking', 'broke']

При выполнении даного задания *было пролито много крови и слез* и было испробовано несколько способов улучшения работы Леска:
1. использование defenition+examples
2. нормалзиация текста (спойлер - nltk ужасно лемматизирует, но узнала я об этом уже слишком поздно. Он автоматически лемматизирует все под существительные, если бы я заинтересовалась его работой раньше, то написала бы еще функцию, которая пос-теггила, а потом уже лемматизировала в зависимости от пос-тега, но было уже поздно... в общем, я вставила небольшой костыль для брейка - сделала строчку, где все варианты брейка переделываются в брейк в norm_token)
3. увеличение окна words_in_context, потому что 3 слова может быть маловато

Я не сразу прочитала, что нужно было оставлять все попытки, простите, пожалуйста! Поэтому здесь одна большая попытка.

У нас есть три функции:

1. **norm_token** - лемматизирует (не очень), токенизирует и убирает стопслова
2. **get_words_in_context** увеличили окно для контекстов слов
3. **lesk** - главный герой программы, где у нас не только определения, но и примеры для слова

In [602]:
def norm_token(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word]
    words = [word for word in words if word and word not in stops]
    words = [word for word in words if word and not digits(word)]
    words = [wordnet_lemmatizer.lemmatize(word) for word in words if word]
    words = ['break' if word in breaks else word for word in words if word]
    
    return words

def get_words_in_context(words, window=7):
    words2context = []
    for i in range(len(words)):
        left = words[max(0,i-window):i] 
        right = words[i+1:i+window+1]
        target = words[i]
        words2context.append((target, left+right)) 
    return words2context

def lesk(word, sentence):
    bestsense = 0
    maxoverlap = 0
    
    for i, synset in enumerate(wn.synsets(word)):
        definition = norm_token(synset.definition())
        examples = norm_token(' '.join(synset.examples()))
        def_ex = set(definition + examples)
        sentence = set(sentence)
        overlap = len(def_ex & sentence)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
            
    return bestsense

Функция дизамбигуации

In [603]:
def disambiguation(corpus):
    dis_corpus = []

    for text in corpus:
        dis_text = []
        words_in_context = get_words_in_context(text)

        for word, context in words_in_context:
            nsense = lesk(word, context)

            # если смысл не нулевой - добавим индекс смысла к токену
            if nsense > 0:
                dis_text.append(word + '_' + str(nsense))
            else:
                dis_text.append(word)

        dis_corpus.append(dis_text)
    return(dis_corpus)

Достаем предложения с break. Всего у нас 1198 строки, но это пришлось извернуться и достать все возможные брейки, потому что nltk очень неудачно отлемматизировался.

In [604]:
corpus = [line for line in open('corpus_eng.txt', 'r', encoding = 'utf-8')]

corpus_norm = [norm_token(line) for line in corpus]

In [605]:
all_contexts = []

for line in corpus_norm:
    if 'break' in line:
        all_contexts.append(line)

print(len(all_contexts))

1198


In [606]:
contexts = []
contexts = random.sample(all_contexts, 10)

len(contexts)

10

In [607]:
print(contexts)

[['communist', "party's", 'mouthpiece', "people's", 'daily', 'warned', 'editorial', 'thursday', 'china', 'must', 'break', 'monopoly', 'core', 'technology', 'standard', 'remain', 'untethered', 'country', 'technology', 'supply', 'chain'], ['world', 'number', 'break', 'karlovic', 'oldest', 'player', 'feature', 'davis', 'cup', 'single', 'match', 'since', 'australian', 'norman', 'brooke', 'four', 'time', 'help', 'argentina', 'banish', 'memory', 'losing', 'final', 'appearance'], ['bridgeport', 'man', 'sentenced', 'southern', 'ct', 'armed', 'robbery', 'spree', 'updated', 'pm', 'tuesday', 'november', 'hartford', 'conn', 'ap', 'bridgeport', 'man', 'sentenced', 'decade', 'federal', 'prison', 'participating', 'armed', 'robbery', 'spree', 'southern', 'connecticut', 'u.s', 'attorney', 'deirdre', 'daly', 'say', 'derrick', 'gilliam', 'sentenced', 'tuesday', 'bridgeport', 'year', 'one', 'month', 'prison', 'conspiracy', 'affect', 'commerce', 'robbery', 'followed', 'supervised', 'release', 'prosecutor',

Здесь мы достаем все СЕМЬДЕСЯТ ПЯТЬ значений слова break (такое количество значений немного усложняет задачу дизамбигуации, честно говоря, поэтому леск будет практически нереально заставить работать качественно)

In [608]:
word = 'break'

defin = { str(indx+1) : str(i.definition()) for indx, i in enumerate(wn.synsets(word))}
    
defin

{'1': 'some abrupt occurrence that interrupts an ongoing activity',
 '2': 'an unexpected piece of good luck',
 '3': "(geology) a crack in the earth's crust resulting from the displacement of one side with respect to the other",
 '4': 'a personal or social separation (as between opposing factions)',
 '5': 'a pause from doing something (as work)',
 '6': 'the act of breaking something',
 '7': 'a time interval during which there is a temporary cessation of something',
 '8': 'breaking of hard tissue such as bone',
 '9': 'the occurrence of breaking',
 '10': 'an abrupt change in the tone or register of the voice (as at puberty or due to emotion)',
 '11': 'the opening shot that scatters the balls in billiards or pool',
 '12': '(tennis) a score consisting of winning a game when your opponent was serving',
 '13': 'an act of delaying or interrupting the continuity',
 '14': 'a sudden dash',
 '15': 'any frame in which a bowler fails to make a strike or spare',
 '16': 'an escape from jail',
 '17': '

Вычленяем правильные значения

Есть гигантская проблема - у большей части примеров (я делала рандом 10 раз) все равно будет метафорическое или частично метафорическое значение. Да, у break целых 75 значений, но даже из них сложно вычленить правильные для метафор. Посмотрим, что получится. Оценивать правильность будем путем сравнения значений, записанных у нас в словаре, со значениями, который будут выдаваться.

In [609]:
result = disambiguation(contexts)

all_breaks = []

print(result)

for sent in result:
    for word in sent:
        if 'break' in word:
            all_breaks.append(re.sub('\D', '', word))
            
print(all_breaks)

[['communist', "party's", 'mouthpiece', "people's", 'daily', 'warned', 'editorial', 'thursday', 'china', 'must', 'break', 'monopoly', 'core', 'technology', 'standard', 'remain', 'untethered', 'country', 'technology', 'supply', 'chain'], ['world', 'number', 'break', 'karlovic', 'oldest', 'player_3', 'feature', 'davis_4', 'cup', 'single', 'match_16', 'since', 'australian', 'norman_1', 'brooke', 'four', 'time', 'help_8', 'argentina', 'banish', 'memory', 'losing', 'final', 'appearance'], ['bridgeport', 'man_1', 'sentenced', 'southern', 'ct', 'armed_2', 'robbery', 'spree', 'updated', 'pm', 'tuesday', 'november', 'hartford', 'conn', 'ap', 'bridgeport', 'man_1', 'sentenced', 'decade', 'federal', 'prison', 'participating', 'armed', 'robbery', 'spree', 'southern', 'connecticut', 'u.s', 'attorney', 'deirdre', 'daly', 'say', 'derrick', 'gilliam', 'sentenced', 'tuesday', 'bridgeport', 'year', 'one', 'month', 'prison', 'conspiracy', 'affect', 'commerce', 'robbery', 'followed', 'supervised', 'releas

In [610]:
for br in all_breaks:
    if br in defin.keys():
        print(br+' '+defin[br])
    else:
        print('1'+' '+defin['1'])

1 some abrupt occurrence that interrupts an ongoing activity
1 some abrupt occurrence that interrupts an ongoing activity
30 surpass in excellence
30 surpass in excellence
14 a sudden dash
30 surpass in excellence
2 an unexpected piece of good luck
30 surpass in excellence
2 an unexpected piece of good luck
19 render inoperable or ineffective
29 fail to agree with; be in violation of; as of rules or patterns
29 fail to agree with; be in violation of; as of rules or patterns


После не-знаю-какой-попытки исправить результаты и на 610 ячейке я немного утомилась и решила остановиться с тем, что есть. Смотрим:
0. В принципе, break monopoly core - это прерывание какой-то активити, 1 балл.
1. Значение не очень ясно из имеющегося контекста, но точно не связано с прерыванием деятельности, 0 баллов.
2. Все ломает breaking news, для улучшения алгоритма его можно почистить. Один из тех случаев, когда очевидно, что метафоричность текста Леску не ясна. 0 баллов.
3. Sudden dash здесь не происходит, но ясно, что он возможно стриггерился на слово crisis. 0 баллов
4. Сломанный стереотип не совсем подходит под определение surpass in excellence. 0 баллов
5. Вау, оно подошло. Действительно, удача в игре. 1 балл.
6. Ох уже мне эти breaking news, надоели. 0 баллов.
7. Похоже на правду, по крайней мере, по слову sufficient он определил good luck. 1 балл. И еще готова дать 100 баллов за правильное определение break_19 во второй части текста - там действительно что-то неработающее/неисправное, проблемы с зубочисткой или что-то вроде того. 1 балл.
8. Мимо, среагировал на что-то в предложении, связанное с несогласием - предложение действительно про соперничество двух команд, можно понять. 1 балл.
9. Слишком мимо, тут побитый рекорд, а не несогласие. 0 баллов

В общем, Леск сработал не очень удачно, но куда лучше (до этого больше 0 он мне не выдавал ни при каких попытках), чем раньше. Использовалась нормализация, чистка стоп-слов, убирание чисел, увеличение окна. Результат - 5/12 были угаданы (относительно, но все же).
Можно улучшить его работу немного, если сделать более качественную лемматизацию, чем есть сейчас (функцию, где нлтк лемматизирует части речи после пос-теггинга, он похоже на это подбит, куда ему до нашего пайморфи).